# Pokemon



![](https://geekandsundry.com/wp-content/uploads/2017/06/rsz_landscape-1456483171-pokemon2.jpg)




In [ ]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))



# Pokemon dataset için gerekli kütüphaneler

In [ ]:
import matplotlib.pyplot as plt #gorselestirmek icin kullandigimiz kutuphane
import seaborn as sns #gorselestirmek icin kullandigimiz kutuphane(isi haritasi)
from sklearn import preprocessing #Veri Normalleştirme icin kullanilan kutuphane
from sklearn import  model_selection   #verileri ayirmak icin kullandigimiz kutuphane test,egitim( cross_validation)

# Csv import ediyoruz

In [ ]:
df=pd.read_csv(r'../input/pokemon.csv') 

# Ilk 10 satir getirelim

In [ ]:
df.head(10) 

# Gereksiz olan  # sutununu cikartalım

In [ ]:
df.drop('#', axis = 1, inplace = True)


In [ ]:
df.head(10) #ilk 10 satir

# Son 10 satırı getirelim

In [ ]:
df.tail(10) 

# Satır ve sütun sayısı

In [ ]:
df.shape

# Bellek kullanımı ve veri türleri
# İnfo kısmı bize verimiz hakkında genel bilgileri verir.

In [ ]:
df.info()

# Basit istatistikler(ortalama,standart sapma, min,max, çeyrekler(quartile(q1=%25,q2=%50,q3=%75))

In [ ]:
df.describe()

#   Özellikler (attributes)

In [ ]:
df.columns

# Filtreleme ve Sıralama 
# Attack 300 dan kucuk olup Defense  200 den yüksek    alanları seç  Attack gore  sirala

In [ ]:
df[(df['Defense']>200) & (df['Attack']<300)].sort_values('Attack', axis=0, ascending=False)  


# Apply fonksiyonu kullanarak Mevcut özniteliklerden yeni bir öznitelik oluşturuyoruz.   

In [ ]:
def Can(hp):
   
  return (hp>100)

df['YuksekCan'] = df['HP'].apply(Can)
df.head(20)

# Sayisal niteliklerin ortalamasi

In [ ]:
df.mean(axis=0,skipna=True)

# Nitelik tipleri

In [ ]:
df.dtypes

# Korelasyon  1(dogru ilişki),-1(ters ilişki) yakınsa güçlü,0 yakınsa aralarındaki  ilişki zayıf anlamındadır 

In [ ]:
df2=df #dataframe kopyaladik
df2.corr() # korelasyon 

# Seaborn ısı haritası amacı korelasyonu bize renkli bir şekilde göstermeye yarıyor
#  Renk ne kadar açık  olursa ilişki o kadar güçlüdür. Renk nekadar koyu olursa aralarındaki ilişki o kadar zayıftır.

In [ ]:
#correlation map
f,ax=plt.subplots(figsize=(12,12))
sns.heatmap(df2.corr(),annot=True,linewidths=.5,fmt='.2f',ax=ax)
plt.show() # 12 12 lik isi haritasi bize corelasyon renkli bir sekilde gosteriyor
#korelasyon tablosu,annot=içindeki sayıların görünürlüğü,linewidths=çerçeve fmt sondan iki basamak

# Korelasyonları yüksek olan 2 tane öznitelik için Scatter Plot

In [ ]:
df2['YuksekCan']=df2['YuksekCan'].astype('float') #YuksekCan niteligi float tipine cevirdik
df2.plot(kind='scatter', x='HP', y='YuksekCan',alpha = 0.5,color = 'red')
plt.xlabel('HP')             
plt.ylabel('YuksekCan')
plt.title('HP YuksekCan Scatter Plot')
df2['YuksekCan']=df2['YuksekCan'].astype('bool') #generation niteligi bool tipine cevirdik

# 	Histogramın   amacı sıklıkları bize renkli bir şekilde göstermeye yarıyor

In [ ]:



df2["Attack"].plot(kind="hist",color="blue",bins=30,grid=True,alpha=0.4,label="Attack",figsize=(18,8))
plt.legend()
plt.xlabel("Attack")
plt.ylabel("sıklık")
plt.title("Attack Sıklığı")
plt.show()


# Ön İşleme  İşlemleri

# Eksik değer var mı ?

In [ ]:
df2.isnull().sum() #Datamız içerisinde tanımlanmamış değerler 

 # Toplam ne kadar eksik deger var ?

In [ ]:
df2.isnull().sum().sum()  #Datamız içerisinde toplam tanımlanmamış değerler 

 # Eksik degerleri yüzdelik olarakta görelim 

In [ ]:
def eksik_deger_tablosu(df): 
    mis_val = df.isnull().sum() #eksik degerler
    mis_val_percent = 100 * df.isnull().sum()/len(df) #eksik degerlerin yuzdelik dilimi
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1) #tabloda birlestir
    mis_val_table_ren_columns = mis_val_table.rename(
    columns = {0 : 'Eksik Değerler', 1 : '% Değeri'})
    return mis_val_table_ren_columns

eksik_deger_tablosu(df2)

 # Eksik olan degerler  silinebilir  ama pek tercih edilmez bunu yerine sayısal veri ise o öznitelik  değerlerini ortalaması  ile doldurulabilir.Dataki herhangi bir kolonda çok eksik veri varsa o zaman o kolon silinebilir. String veri ise belirli bir string  ifade ile doldurulabilir.

# Datamızındaki sayısal değerler için eksik değerleri yok etmek yerine onlara ortalama değerler vererek korelasyonu bozmayalım

In [ ]:
df3=df2
df3['HP'] = df3['HP'].transform(lambda x: x.fillna(x.mean()))
df3['Attack'] = df3['Attack'].transform(lambda x: x.fillna(x.mean()))
df3['Defense'] = df3['Defense'].transform(lambda x: x.fillna(x.mean()))
df3['Sp. Atk'] = df3['Sp. Atk'].transform(lambda x: x.fillna(x.mean()))
df3['Sp. Def'] = df3['Sp. Def'].transform(lambda x: x.fillna(x.mean()))
df3['Speed'] = df3['Speed'].transform(lambda x: x.fillna(x.mean()))




# Datamızındaki string degerlerde  null degerler icin  empty yazalım.

In [ ]:
df3['Type 2'].fillna('empty',inplace=True) #empty ile doldur
df3['Name'].fillna('empty',inplace=True) #empty ile doldur

df3.head(10)


# Datamızındaki speed  özniteliğini normalleştirelim. 

In [ ]:
#Veri Normalleştirme

#Speed özniteliğini normalleştirmek için
x = df3[['Speed']].values.astype(float) #float tipine donüştürüyoruz

#Normalleştirme için MinMax normalleştirme yöntemini kullanıyoruz.
#speed niteligini 0-1 arasinda degerler indirgiyoruz.
min_max_scaler = preprocessing.MinMaxScaler((0,1))
x_scaled = min_max_scaler.fit_transform(x)
df3['Speed'] = pd.DataFrame(x_scaled)

df3



# BoxPlot ile Aykırı Uç değerlerini görelim

In [ ]:

sns.boxplot(x=df3['Attack'])
plt.show()

#  Uç değerlerimiz

In [ ]:
den = np.percentile(df3.Attack, [0, 100])
den

# Uç değerler arasindaki veriler

In [ ]:
new_df = df3[(df3.Attack > den[0]) & (df3.Attack < den[1])] 
new_df

#  Lengendary ,YuksekCan gibi tru false olan degerleri 1 ve 0 degerlerine  cevirdik.

In [ ]:
df3['Legendary'].replace(False, 0, inplace=True) #burda legendary true false dan 1,0 konumuna getiriyoruz
df3['YuksekCan'].replace(True, 1, inplace=True) #burda YuksekCan true false dan 1,0 konumuna getiriyoruz
df3

# 	Histogram Ayrıntılı   

In [ ]:
num_bins = 10
df3.hist(bins=num_bins, figsize=(20,15))

# Sayisal olmayan icerikleri çıkaralım model eğitimi için

In [ ]:
df4=df3
df4.drop('Name', axis = 1, inplace = True)
df4.drop('Type 1', axis = 1, inplace = True)
df4.drop('Type 2', axis = 1, inplace = True)

df4

# Data eşit şekilde parçalama icin Legendary true ve false sayislarini görelim

In [ ]:
data1 = df3[df3['Legendary']==1]
data2 = df3[df3['Legendary']==0]
data1.shape #lengendary  olanlar(true)

In [ ]:
data2.shape  #lengendary olmayanlar(false)

# Her iki sinifta da eşit sayida örnek olmasını istiyoruz

In [ ]:
data = data1.append(data2[:65])

# Son veri kümesi

In [ ]:
data.shape

# Eğitim  ve test verisini parçaliyoruz  ( Egitim=80% , Test=20%)

In [ ]:
X = data.ix[:, data.columns != 'Legendary']
Y = data['Legendary']

# Test ve eğitim dataları ayiralim.Random state belli bir oranda bölmemesi saglar

In [ ]:
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y, test_size=0.2, random_state=0)







# Veri seti üzerinde normalizasyon yaparak -1,1 arasında degerlere indirgiyoruz. preprocessing paketi içerisindeki MinMaxScaler fonksiyonunu kullanarak gerçekleştiriyoruz. 

In [ ]:
X_train = X_train.astype('float32') #float tipine ceviriyoruz
X_test = X_test.astype('float32') #float tipine ceviriyoruz
#ölçeklendirme
scaler = preprocessing.MinMaxScaler((-1,1))
scaler.fit(X)
XX_train = scaler.transform(X_train.values)
XX_test  = scaler.transform(X_test.values)
YY_train = Y_train.values 
YY_test  = Y_test.values

# Sınıflandırma Modellerine Ait Kütüphaneler

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier

# Kullanacagimiz machine learning algoritmalarini modelleri models e ekledik

In [ ]:
models = []
models.append(('Naive Bayes', GaussianNB()))
models.append(('Logistic Regression', LogisticRegression()))
models.append(('RandomForestClassifier', RandomForestClassifier())) 

# Modelleri test edip Accuracy ve Confusion matrisini  görelim.

In [ ]:
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from sklearn.feature_selection import RFE

# Modelleri test edelim
for name, model in models:
    model = model.fit(X_train, Y_train)
    Y_pred = model.predict(X_test)
    
    #Accuracy değeri gör
    print("%s -> ACC: %%%.2f" % (name,metrics.accuracy_score(Y_test, Y_pred)*100))
    
    #Confusion matris görmek için aşağıdaki kod satırlarını kullanabilirsiniz   
    report = classification_report(Y_test, Y_pred)
    print(report)

# Oluşturduğumuz eğitim ve test veri setlerini kullanarak, modelleri eğittik ve onları test ettik


